##### Script Deployment: Batch Deployment with scoring script

**Workflow**
1. Initialize Workspace & creat workspace handle
2. Initialize Compute Cluster & Environment 
3. Get referrence of the register model 
4. Configure & Create Endpoint
5. Create a Scoring script
6. Configure & Create Deployment
7. Set Deployment as Default
8. Test the Deployment
9. Clean up the resources

##### Step 1: Initialize Workspace and Create Workspace handle

In [1]:
from azureml.core import Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Initialize  workspace
ws = Workspace.from_config()  

# Get a handle to the workspace
credential = DefaultAzureCredential()  # authenticate
ml_client = MLClient( credential=credential,
                      subscription_id=ws.subscription_id,
                      resource_group_name=ws.resource_group,
                      workspace_name=ws.name,
                    )


##### Step 2: Initialize or Get Compute Cluster

In [2]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
compute = "ML-Pipeline-Cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(compute)
    print(f"You already have a cluster named {compute}, we'll reuse it as is.")

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=compute,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=300,
        tier="Dedicated",
    )
    print(f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}")
    
    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

You already have a cluster named ML-Pipeline-Cluster, we'll reuse it as is.


##### Environment

In [3]:
import os
from azure.ai.ml.entities import Environment

custom_env_name  = "ENV-SDKv2"
# dependencies_dir = '../dependencies'
# env = Environment( name=custom_env_name,
#                    description="Evironment for python SDKv2 Execution",
#                    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
#                    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
#                  )
# env = ml_client.environments.create_or_update(env)

# GET ENVIRONMENT
# use 'label' parameter to get latest environment for example label='latest'
# use 'version' parameter to get specific version environment, for example version=2
env = ml_client.environments.get(name=custom_env_name, label='latest') 

print(f"Environment with name {env.name} is registered to workspace, the environment version is {env.version}")

Environment with name ENV-SDKv2 is registered to workspace, the environment version is 9


##### Step 3: Get referrence to Registered Model

In [4]:
model_name='pima_model_SDKv2_04'
model = ml_client.models.get(name=model_name, label="latest")

##### Step 4: Configure and Create Batch Endpoint

Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `description`- Description of the endpoint.

In [5]:
from azure.ai.ml.entities import BatchEndpoint

# configure
endpoint_name = "pima-batch-endpoint-sdk-v2"
endpoint = BatchEndpoint( name=endpoint_name,
                          description="Sample Batch Endpoint using SDKv2"
                          )

# create
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

##### Step 5: Create a Scoring Script
we'll use scoring script in our next step, Batch Deployment

##### Step 6: Configure & Create a batch deployment


In [6]:
# imports
from azure.ai.ml.entities import ( 
                                  ModelBatchDeployment,
                                  ModelBatchDeploymentSettings,
                                  BatchRetrySettings,
                                  CodeConfiguration
                                )
from azure.ai.ml.constants import BatchDeploymentOutputAction

# configure
deployment_name = 'script-deployment-batch'
deployment = ModelBatchDeployment(
    name=deployment_name,
    description="pima diabetes classifier with a custom scoring script",
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(code="../src", scoring_script="pima_scoreBatchEndpoint_SDKv2.py"),
    compute=compute,
    settings=ModelBatchDeploymentSettings(
        instance_count=2,
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
        logging_level="info",
    ),
)

# create deployment
ml_client.batch_deployments.begin_create_or_update(deployment).result()

Class ModelBatchDeploymentSettings: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ModelBatchDeployment: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.0 MBs): 100%|██████████| 4837/4837 [00:00<00:00, 36997.38it/s]




BatchDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'pima-batch-endpoint-sdk-v2', 'type': None, 'name': 'script-deployment-batch', 'description': 'pima diabetes classifier with a custom scoring script', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspaces/us_azure/batchEndpoints/pima-batch-endpoint-sdk-v2/deployments/script-deployment-batch', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/instancenotebook/code/Users/uj_az/PIMA/03-Deploying-Batch-Endpoint/SDKv2/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f6b5d4de980>, 'serialize': <msrest.serialization.Serializer object at 0x7f6b5d4de8f0>, 'model': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspac

##### Step 7: Set the deployment as Default Deployment

In [7]:
endpoint = ml_client.batch_endpoints.get(endpoint.name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()
print(f"The default deployment is {endpoint.defaults.deployment_name}")



The default deployment is script-deployment-batch


##### Step 8: Testing the deployment
- Get Test Data
- create input for deployment
- Invoke the deployment


In [23]:
# #input = 
# Input(type=AssetTypes.URI_FILE, path=test_data.id)

In [24]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
# Test Data
dataset_name ='test_pima_data_typeFile_SDKv2'
test_data =  ml_client.data.get(name = dataset_name, label = "latest")

# creating an input for the deployment
input = Input(type=AssetTypes.URI_FILE, path=test_data.id)

# Invoke the deployment
#job = ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)
job = ml_client.batch_endpoints.invoke(deployment_name=deployment.name, 
                                       endpoint_name=endpoint.name, 
                                       input=input
                                      )
# Get details of invoked job
ml_client.jobs.get(job.name)

AttributeError: 'ModelBatchDeployment' object has no attribute 'namae'

{'type': 'uri_file', 'path': '/subscriptions/ba5d6a04-af22-45ea-bc5a-946ef1c32949/resourceGroups/us_azure_practice/providers/Microsoft.MachineLearningServices/workspaces/us_azure/data/test_pima_data_typeFile_SDKv2/versions/1'}

##### Step 9: Clean up resources

In [ ]:
ml_client.batch_endpoints.begin_delete(endpoint_name).result()

....

In [15]:

1+!

SyntaxError: invalid syntax (3679708973.py, line 1)